In [ ]:
# Reshape metadata to tidy format
import pandas as pd
df = pd.read_csv("wdi_series_metadata.csv", encoding='latin-1')
tidy_df = df.melt(
    id_vars=["Country Name", "Country Code", "Series Name", "Series Code"],
    var_name="Year",
    value_name="Value"
)
tidy_df["Year"] = tidy_df["Year"].str.extract(r"(\d{4})").astype(int)
tidy_df["Value"] = pd.to_numeric(tidy_df["Value"], errors="coerce")
tidy_df.to_csv("wdi_tidy.csv", index=False)

In [ ]:
# Profile data by indicator
df = pd.read_csv("wdi_tidy.csv")
profile = df.groupby(['Series Code', 'Series Name']).agg(
    total_records=('Value', 'size'),
    missing_count=('Value', lambda x: x.isna().sum()),
    mean_value=('Value', 'mean'),
    min_value=('Value', 'min'),
    max_value=('Value', 'max')
).reset_index()
profile['missing_pct'] = profile['missing_count'] / profile['total_records'] * 100
profile.to_csv("indicator_profile.csv", index=False)

In [ ]:
# Filter indicators and prepare final dataset
profile = pd.read_csv("indicator_profile.csv")
filtered = profile[profile["missing_pct"] < 25]
filtered.to_csv("indicators_under_25pct_missing.csv", index=False)

selected = ["DT.DOD.DECT.CD","FI.RES.TOTL.CD","FP.CPI.TOTL.ZG","FR.INR.LEND","NY.GDP.MKTP.CD","NY.GDP.PCAP.CD"]
df = pd.read_csv("wdi_tidy.csv")
filtered_df = df[df["Series Code"].isin(selected)]
filtered_df.to_csv("wdi_filtered.csv", index=False)

In [ ]:
# Build dimension and fact tables
df = pd.read_csv("wdi_filtered.csv")
dim_country = df[['Country Name','Country Code']].drop_duplicates().reset_index(drop=True)
dim_country['Country ID'] = dim_country.index + 1
dim_indicator = df[['Series Name','Series Code']].drop_duplicates().reset_index(drop=True)
dim_indicator['Indicator ID'] = dim_indicator.index + 1
dim_time = pd.DataFrame({'Year': sorted(df['Year'].unique())})
dim_time['Time ID'] = dim_time.index + 1

fact = df.merge(dim_country, on=['Country Name','Country Code']) \
         .merge(dim_indicator, on=['Series Name','Series Code']) \
         .merge(dim_time, on='Year')
fact_table = fact[['Country ID','Indicator ID','Time ID','Value']]

dim_country.to_csv("dim_country.csv", index=False)
dim_indicator.to_csv("dim_indicator.csv", index=False)
dim_time.to_csv("dim_time.csv", index=False)
fact_table.to_csv("fact_indicator_value.csv", index=False)

## SQL DDL and DML

In [ ]:
-- Create tables
CREATE TABLE dim_country (
    country_id SERIAL PRIMARY KEY,
    country_name TEXT,
    country_code TEXT
);

CREATE TABLE dim_indicator (
    indicator_id SERIAL PRIMARY KEY,
    indicator_name TEXT,
    indicator_code TEXT
);

CREATE TABLE dim_year (
    year INT PRIMARY KEY
);

CREATE TABLE fact_economic_indicators (
    fact_id SERIAL PRIMARY KEY,
    country_id INT REFERENCES dim_country(country_id),
    indicator_id INT REFERENCES dim_indicator(indicator_id),
    year_id INT REFERENCES dim_year(year),
    value NUMERIC
);

-- Populate dimensions
INSERT INTO dim_country (country_name, country_code)
SELECT DISTINCT country_name, country_code FROM world_bank_data;
INSERT INTO dim_indicator (indicator_name, indicator_code)
SELECT DISTINCT series_name, series_code FROM world_bank_data;
INSERT INTO dim_year (year)
SELECT DISTINCT year FROM world_bank_data;

-- Populate fact table
INSERT INTO fact_economic_indicators (country_id, indicator_id, year_id, value)
SELECT dc.country_id, di.indicator_id, dy.year_id, wbd.value
FROM world_bank_data wbd
JOIN dim_country dc ON wbd.country_name = dc.country_name
JOIN dim_indicator di ON wbd.series_name = di.indicator_name
JOIN dim_year dy ON wbd.year = dy.year;